\section{Transverse field Ising model(TFIM)}

Model Hamiltonian:
\begin{equation}
    H = \sum_{<ij>} -J Z_i Z_j -\sum_i \Gamma X_i
\end{equation}
where in $X$ and $Z$ are Pauli matirces.


1. Normalize of a vector (inner product)

In [1]:
# packages
using LinearAlgebra
using Random; Random.seed!()
using BenchmarkTools
using Optim
using Zygote

In [2]:
include("setup_struct.jl")
include("toolfunctions.jl")

symmetrize (generic function with 1 method)

In [3]:
# Products
function myprod(O::cMPO, S::cMPS)
    Oi = Matrix(1.0I,size(O.Q))
    Si = Matrix(1.0I,size(S.Q))
    Q = kron(Oi , S.Q) + kron(O.Q , Si) + kron(O.L , S.R)
    R = kron(O.R , Si) + kron(O.P , S.R)
    return cMPS(Q, R)
end

function myinnerprod(sl::cMPS, sr::cMPS, β::Real)
    li = Matrix(1.0I,size(sl.Q))    
    ri = Matrix(1.0I,size(sr.Q))    
    K = kron(li , sr.Q) + kron(sl.Q , ri) + kron(sl.R , sr.R)
    vals = eigvals(K)
    res = 0.
    for i=1:length(vals)
        res += exp(-β*vals[i])
    end
    return real(res)
end

# Physical
function F(ψ::cMPS, W::cMPO, β::Real)
    Hψ = myprod(W,ψ)
    res = log(myinnerprod(ψ, Hψ ,β))- log(myinnerprod(ψ,ψ,β))
    return -1/β * res
end 

function difference(ψ1::cMPS, ψ2::cMPS; β=1)
    res = myinnerprod(ψ1,ψ1,β) + myinnerprod(ψ2,ψ2,β)
    res -= myinnerprod(ψ2,ψ1,β) + myinnerprod(ψ1,ψ2,β)
    return res
end

difference (generic function with 1 method)

In [4]:
"""
optimize difference
"""
function OptimDiff(x::Array{Float64,3})
    ψ = cMPS(x[:,:,1], x[:,:,2])
    return difference(ψ,Hψ)
end

function OptimDiff!(gx::Array{Float64,3}, x::Array{Float64,3})
    ψ = cMPS(x[:,:,1], x[:,:,2])
    grad = gradient(ψ -> difference(ψ,Hψ), ψ)[1]
    (r,c) = size(grad.Q)
    for i = 1:r, j = 1:c
        gx[i,j,1] = grad.Q[i,j]
        gx[i,j,2] = grad.R[i,j]
    end
end

OptimDiff! (generic function with 1 method)

In [5]:
"""
optimize free energy
"""
function OptimF(x::Array{Float64,3})
    ψ = cMPS(x[:,:,1], x[:,:,2])
    return F(ψ,W,β)
end

function OptimF!(gx::Array{Float64,3}, x::Array{Float64,3})
    ψ = cMPS(x[:,:,1], x[:,:,2])
    grad = gradient(ψ -> F(ψ, W, β), ψ)[1]
    (r,c) = size(grad.Q)
    for i = 1:r, j = 1:c
        gx[i,j,1] = grad.Q[i,j]
        gx[i,j,2] = grad.R[i,j]
    end
end

OptimF! (generic function with 1 method)

In [8]:
"""Test"""
J = 1.0; Γ = 1.0
T = 1.
β = 1 / T
X = [0. 1.; 1. 0.]
Z = [1. 0.; 0. -1.]
W = cMPO(Γ*X, √J*Z, √J*Z, zeros(2,2))
fac = 1.
Q = fac * rand(2,2) |> symmetrize
R = fac * rand(2,2) |> symmetrize
ψ = cMPS(Q,R)

Hψ = myprod(W, ψ)
x = toarray(ψ)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.104172  0.952396
 0.952396  0.424975

[:, :, 2] =
 0.657828  0.460432
 0.460432  0.988196

In [9]:
op = optimize(OptimF, x, LBFGS(),Optim.Options(show_trace=true))

Iter     Function value   Gradient norm 
     0    -1.724698e+00     1.091674e+00
 * time: 0.027743816375732422
     1    -9.104095e+02     1.204247e+00
 * time: 0.8723340034484863
     2    -9.104095e+02     1.204247e+00
 * time: 0.8997688293457031
     3    -9.104589e+02     2.527224e+02
 * time: 0.9074459075927734
     4    -9.121558e+02     1.204321e+00
 * time: 0.9575679302215576
     5    -9.121558e+02     1.204321e+00
 * time: 0.9763607978820801
     6    -9.121558e+02     1.204321e+00
 * time: 1.0006158351898193
     7    -9.121558e+02     1.204321e+00
 * time: 1.0322468280792236


 * Status: success

 * Candidate solution
    Final objective value:     -9.121558e+02

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.42e-14 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.17e-16 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.20e+00 ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    7
    f(x) calls:    466
    ∇f(x) calls:   466


In [10]:
op = optimize(OptimF, OptimF!, x, LBFGS(),Optim.Options(show_trace=true))

Iter     Function value   Gradient norm 
     0    -1.724698e+00     1.328573e+00
 * time: 0.0001010894775390625


LoadError: DomainError with -0.0478083308763824:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [11]:
fac = 1.
Q1 = fac * rand(2,2) |> symmetrize
R1 = fac * rand(2,2) |> symmetrize
ψ1 = cMPS(Q,R)
x1 = toarray(ψ)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.104172  0.952396
 0.952396  0.424975

[:, :, 2] =
 0.657828  0.460432
 0.460432  0.988196

In [12]:
op1 = optimize(OptimDiff, x1, LBFGS(),Optim.Options(show_trace=true))

Iter     Function value   Gradient norm 
     0     3.328945e+01     2.051816e+01
 * time: 0.0001220703125
     1   -3.282146e+306    1.641216e+306
 * time: 0.12431192398071289


 * Status: failure (line search failed)

 * Candidate solution
    Final objective value:     -3.282146e+306

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.64e+306 ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    2
    f(x) calls:    82
    ∇f(x) calls:   82


In [13]:
op1 = optimize(OptimDiff, OptimDiff!, x1, LBFGS(),Optim.Options(show_trace=true))

Iter     Function value   Gradient norm 
     0     3.328945e+01     2.051816e+01
 * time: 0.00011706352233886719
     1     3.328945e+01     2.051816e+01
 * time: 2.683943033218384
     2   -1.797693e+308    1.500154e+308
 * time: 2.8569419384002686


 * Status: failure (line search failed)

 * Candidate solution
    Final objective value:     -1.797693e+308

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.50e+308 ≰ 1.0e-08

 * Work counters
    Seconds run:   3  (vs limit Inf)
    Iterations:    3
    f(x) calls:    106
    ∇f(x) calls:   106
